In [6]:
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from functools import reduce
import operator
import json
import re
from geopy.geocoders import Nominatim

In [125]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [7]:
df = pd.read_csv('df_final_clean3.csv', index_col=0)
df.dropna(how = "all", axis = 0, inplace = True)

In [8]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3
indice,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España"


In [9]:
import ast

In [10]:
def ast_literal (x):
    try:
        return ast.literal_eval(x)
    except:
        return x

In [11]:
df.coordinates = df.coordinates.apply(ast_literal)

In [12]:
def turn_around(x):
    b = (x[1], x[0])
    return list(b)

In [13]:
df["coordinates_v2"] = df.coordinates.apply(turn_around)

In [14]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2
indice,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España","[-3.6142713, 40.0395885]"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España","[-3.6142713, 40.0395885]"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España","[-3.4252641, 40.1384846]"


In [15]:
type(df.coordinates[0])

tuple

In [16]:
type(df.coordinates_v2[0])

list

In [17]:
df.dtypes

name              object
address           object
type              object
ranking           object
price             object
direccion         object
comunidad         object
street            object
street_2          object
coordinates       object
street_3          object
coordinates_v2    object
dtype: object

In [44]:
def type_point(x):
    return {"type":"Point", "coordinates": [x[1], x[0]]}

In [45]:
df["type_point"] = df["coordinates"].apply(type_point)

In [46]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
indice,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4..."
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ..."


## Exportar a Mongo

In [47]:
client = MongoClient("localhost:27017")

In [48]:
db = client.get_database("PROJECT")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

In [49]:
collection = db["Restaurants"]

In [50]:
collection.create_index([("type_point_", "2dsphere")])

'type_point__2dsphere'

In [51]:
data = df.to_dict(orient='records')

In [53]:
collection.insert_many(data)

In [127]:
ubicacion = input("¿Dónde estás?: ")
print(ubicacion)

¿Dónde estás?: Calle Feijoo 10, Madrid
Calle Feijoo 10, Madrid


In [129]:
tipo = input("¿Qué tipo de comida te apetece?: ")
print(tipo)

¿Qué tipo de comida te apetece?: Mediterránea
Mediterránea


In [130]:
distance = int(input("¿Cuántos metros quieres desplazarte?: "))
print(distance)

¿Cuántos metros quieres desplazarte?: 500
500


In [131]:
def get_coordenadas(address):
    """
    Esta función saca las coordenadas de la ciudad que le pases.
    Args: una ciudad (string).
    Return: Las coordeandas de la ciudad que le paso como argumento (latitud y longitud).
    """
    geolocator = Nominatim(user_agent="Ana")
    location = geolocator.geocode(query=address, exactly_one=True)
    localizacion = location[1]
    return [localizacion[1], localizacion[0]]

In [132]:
coordinate = get_coordenadas(ubicacion)

In [133]:
coordinate

[-3.7033519, 40.4347798]

In [145]:
inicial_lat = coordinate[1]
inicial_long =  coordinate[0]

In [134]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [135]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

## Restaurantes a 500 m

In [207]:
restaurants = db.get_collection("Restaurants")

In [211]:
all_restaurants = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}}}

In [212]:
(list(restaurants.find(all_restaurants)))

[{'_id': ObjectId('61b89d2a5a0da3de9425be0f'),
  'name': 'Ostras Pedrín Madrid Chamberí',
  'address': 'Calle del Cardenal Cisneros 39 Esquina, 28010 Madrid España',
  'type': 'Mediterránea',
  'ranking': '#1.272 de 10.409 restaurantes en Madrid',
  'price': '€€ - €€€',
  'direccion': 'Calle del Cardenal Cisneros 39 Esquina',
  'comunidad': 'Madrid España',
  'street': 'Calle del Cardenal Cisneros 39',
  'street_2': 'Calle del Cardenal Cisneros 39, Madrid España',
  'coordinates': [40.4341914, -3.7027635],
  'street_3': 'Calle del Cardenal Cisneros 39, Madrid España',
  'coordinates_v2': [-3.7027635, 40.4341914],
  'type_point': {'type': 'Point', 'coordinates': [-3.7027635, 40.4341914]}},
 {'_id': ObjectId('61b89d2a5a0da3de9425be09'),
  'name': 'La Tagliatella Fuencarral, Madrid',
  'address': 'Calle Fuencarral 127, 28010 Madrid España',
  'type': 'Italiana',
  'ranking': '#9.983 de 10.409 restaurantes en Madrid',
  'price': '€€ - €€€',
  'direccion': 'Calle Fuencarral 127',
  'comunid

In [213]:
df_restaurantes= pd.DataFrame(list(restaurants.find(all_restaurants)))

In [216]:
df_restaurantes

,_id,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
0,61b89d2a5a0da3de9425be0f,Ostras Pedrín Madrid Chamberí,"Calle del Cardenal Cisneros 39 Esquina, 28010 ...",Mediterránea,#1.272 de 10.409 restaurantes en Madrid,€€ - €€€,Calle del Cardenal Cisneros 39 Esquina,Madrid España,Calle del Cardenal Cisneros 39,"Calle del Cardenal Cisneros 39, Madrid España","[40.4341914, -3.7027635]","Calle del Cardenal Cisneros 39, Madrid España","[-3.7027635, 40.4341914]","{'type': 'Point', 'coordinates': [-3.7027635, ..."
1,61b89d2a5a0da3de9425be09,"La Tagliatella Fuencarral, Madrid","Calle Fuencarral 127, 28010 Madrid España",Italiana,#9.983 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Fuencarral 127,Madrid España,Calle Fuencarral 127,"Calle Fuencarral 127, Madrid España","[40.4333131, -3.7041275]","Calle Fuencarral 127, Madrid España","[-3.7041275, 40.4333131]","{'type': 'Point', 'coordinates': [-3.7041275, ..."
2,61b89d2a5a0da3de9425be02,Llantén Veggie Bar,"Calle Cardenal Cisneros 40, 28010 Madrid España",Internacional,#1.384 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Cardenal Cisneros 40,Madrid España,Calle Cardenal Cisneros 40,"Calle Cardenal Cisneros 40, Madrid España","[40.4321377, -3.7021397]","Calle Cardenal Cisneros 40, Madrid España","[-3.7021397, 40.4321377]","{'type': 'Point', 'coordinates': [-3.7021397, ..."
3,61b89d2a5a0da3de9425bdfe,NAP. Neapolitan Authentic Pizza,"Calle Olid 11, 28010 Madrid España",Italiana,#2.349 de 10.409 restaurantes en Madrid,€,Calle Olid 11,Madrid España,Calle Olid 11,"Calle Olid 11, Madrid España","[40.4315985, -3.703331]","Calle Olid 11, Madrid España","[-3.703331, 40.4315985]","{'type': 'Point', 'coordinates': [-3.703331, 4..."
4,61b89d2a5a0da3de9425bdfd,Kalua Helado Artesanal,"Calle Fuencarral 131 Heladería Kalúa, 28010 Ma...",Confiterías/panaderías,N.º 6 de 477 cafeterías y teterías en Madrid,€,Calle Fuencarral 131 Heladería Kalúa,Madrid España,Calle Fuencarral 131,"Calle Fuencarral 131, Madrid España","[40.4315634, -3.7036872]","Calle Fuencarral 131, Madrid España","[-3.7036872, 40.4315634]","{'type': 'Point', 'coordinates': [-3.7036872, ..."
5,61b89d2a5a0da3de9425be0c,Bacira,"Calle del Castillo 16, 28010 Madrid España",Japonesa,#301 de 10.409 restaurantes en Madrid,€€ - €€€,Calle del Castillo 16,Madrid España,Calle del Castillo 16,"Calle del Castillo 16, Madrid España","[40.4337811, -3.6989949]","Calle del Castillo 16, Madrid España","[-3.6989949, 40.4337811]","{'type': 'Point', 'coordinates': [-3.6989949, ..."
6,61b89d2a5a0da3de9425be10,La Espumosa Terraza MADRID,Calle de Santa Engracia 60 Entrada Glorieta de...,Bar,#3.064 de 10.409 restaurantes en Madrid,€€ - €€€,Calle de Santa Engracia 60 Entrada Glorieta de...,Madrid España,Calle de Santa Engracia 60,"Calle de Santa Engracia 60, Madrid España","[40.4353703, -3.6987443]","Calle de Santa Engracia 60, Madrid España","[-3.6987443, 40.4353703]","{'type': 'Point', 'coordinates': [-3.6987443, ..."
7,61b89d2a5a0da3de9425be0e,Kitchen 154 (Vallehermoso),"Calle de Vallehermoso, 36 Mercado de Valleherm...",India,#391 de 10.409 restaurantes en Madrid,€€ - €€€,Calle de Vallehermoso 36 Mercado de Vallehermo...,Madrid España,Calle de Vallehermoso 36,"Calle de Vallehermoso 36, Madrid España","[40.4340111, -3.7081041]","Calle de Vallehermoso 36, Madrid España","[-3.7081041, 40.4340111]","{'type': 'Point', 'coordinates': [-3.7081041, ..."
8,61b89d2a5a0da3de9425bdfc,La Tomata,"Calle Alburquerque 13, 28010 Madrid España",Mexicana,#614 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Alburquerque 13,Madrid España,Calle Alburquerque 13,"Calle Alburquerque 13, Madrid España","[40.4311766, -3.7020589]","Calle Alburquerque 13, Madrid España","[-3.7020589, 40.4311766]","{'type': 'Point', 'coordinates': [-3.7020589, ..."
9,61b89d2a5a0da3de9425be0d,Grosso Napoletano,"Calle de Santa Engracia 48, 28010 Madrid España",Italiana,#657 de 10.409 restaurantes en Madrid,€€ - €€€,Calle de Santa Engracia 48,Mad

In [217]:
df_restaurantes.type.unique()

array(['Mediterránea', 'Italiana', 'Internacional',
       'Confiterías/panaderías', 'Japonesa', 'Bar', 'India', 'Mexicana',
       'Pub con cerveza artesanal'], dtype=object)

## Restaurantes Mediterraneos

In [186]:
rest_mediterraneos = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}},'type': "Mediterránea"}

In [187]:
rest_mediterraneos

{'coordinates_v2': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-3.7033519, 40.4347798]},
   '$maxDistance': 500}},
 'type': 'Mediterránea'}

In [188]:
len(list(restaurants.find(query_restaurants)))

1

In [189]:
df_medit = pd.DataFrame(list(restaurants.find(query_restaurants)))

In [190]:
df_medit

,_id,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
0,61b89d2a5a0da3de9425be0f,Ostras Pedrín Madrid Chamberí,"Calle del Cardenal Cisneros 39 Esquina, 28010 ...",Mediterránea,#1.272 de 10.409 restaurantes en Madrid,€€ - €€€,Calle del Cardenal Cisneros 39 Esquina,Madrid España,Calle del Cardenal Cisneros 39,"Calle del Cardenal Cisneros 39, Madrid España","[40.4341914, -3.7027635]","Calle del Cardenal Cisneros 39, Madrid España","[-3.7027635, 40.4341914]","{'type': 'Point', 'coordinates': [-3.7027635, ..."


## Restaurantes Española

In [200]:
rest_internacional = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}},'type': "Internacional"}

In [201]:
rest_internacional

{'coordinates_v2': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-3.7033519, 40.4347798]},
   '$maxDistance': 500}},
 'type': 'Internacional'}

In [202]:
len(list(restaurants.find(rest_internacional)))

1

In [203]:
df_intern = pd.DataFrame(list(restaurants.find(rest_internacional)))

In [204]:
df_intern

,_id,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
0,61b89d2a5a0da3de9425be02,Llantén Veggie Bar,"Calle Cardenal Cisneros 40, 28010 Madrid España",Internacional,#1.384 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Cardenal Cisneros 40,Madrid España,Calle Cardenal Cisneros 40,"Calle Cardenal Cisneros 40, Madrid España","[40.4321377, -3.7021397]","Calle Cardenal Cisneros 40, Madrid España","[-3.7021397, 40.4321377]","{'type': 'Point', 'coordinates': [-3.7021397, ..."


## Maps

In [239]:
df_restaurantes.head()

,_id,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
0,61b89d2a5a0da3de9425be0f,Ostras Pedrín Madrid Chamberí,"Calle del Cardenal Cisneros 39 Esquina, 28010 ...",Mediterránea,#1.272 de 10.409 restaurantes en Madrid,€€ - €€€,Calle del Cardenal Cisneros 39 Esquina,Madrid España,Calle del Cardenal Cisneros 39,"Calle del Cardenal Cisneros 39, Madrid España","[40.4341914, -3.7027635]","Calle del Cardenal Cisneros 39, Madrid España","[-3.7027635, 40.4341914]","{'type': 'Point', 'coordinates': [-3.7027635, ..."
1,61b89d2a5a0da3de9425be09,"La Tagliatella Fuencarral, Madrid","Calle Fuencarral 127, 28010 Madrid España",Italiana,#9.983 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Fuencarral 127,Madrid España,Calle Fuencarral 127,"Calle Fuencarral 127, Madrid España","[40.4333131, -3.7041275]","Calle Fuencarral 127, Madrid España","[-3.7041275, 40.4333131]","{'type': 'Point', 'coordinates': [-3.7041275, ..."
2,61b89d2a5a0da3de9425be02,Llantén Veggie Bar,"Calle Cardenal Cisneros 40, 28010 Madrid España",Internacional,#1.384 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Cardenal Cisneros 40,Madrid España,Calle Cardenal Cisneros 40,"Calle Cardenal Cisneros 40, Madrid España","[40.4321377, -3.7021397]","Calle Cardenal Cisneros 40, Madrid España","[-3.7021397, 40.4321377]","{'type': 'Point', 'coordinates': [-3.7021397, ..."
3,61b89d2a5a0da3de9425bdfe,NAP. Neapolitan Authentic Pizza,"Calle Olid 11, 28010 Madrid España",Italiana,#2.349 de 10.409 restaurantes en Madrid,€,Calle Olid 11,Madrid España,Calle Olid 11,"Calle Olid 11, Madrid España","[40.4315985, -3.703331]","Calle Olid 11, Madrid España","[-3.703331, 40.4315985]","{'type': 'Point', 'coordinates': [-3.703331, 4..."
4,61b89d2a5a0da3de9425bdfd,Kalua Helado Artesanal,"Calle Fuencarral 131 Heladería Kalúa, 28010 Ma...",Confiterías/panaderías,N.º 6 de 477 cafeterías y teterías en Madrid,€,Calle Fuencarral 131 Heladería Kalúa,Madrid España,Calle Fuencarral 131,"Calle Fuencarral 131, Madrid España","[40.4315634, -3.7036872]","Calle Fuencarral 131, Madrid España","[-3.7036872, 40.4315634]","{'type': 'Point', 'coordinates': [-3.7036872, ..."


In [240]:
def get_latitude (x):
    return x[0]

In [241]:
def get_longitude (x):
    return x[1]

In [243]:
df_restaurantes["latitud"] = df_restaurantes["coordinates"].apply(get_latitude)

In [244]:
df_restaurantes["longitude"] = df_restaurantes["coordinates"].apply(get_longitude)

In [245]:
df_restaurantes

,_id,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point,latitud,longitude
0,61b89d2a5a0da3de9425be0f,Ostras Pedrín Madrid Chamberí,"Calle del Cardenal Cisneros 39 Esquina, 28010 ...",Mediterránea,#1.272 de 10.409 restaurantes en Madrid,€€ - €€€,Calle del Cardenal Cisneros 39 Esquina,Madrid España,Calle del Cardenal Cisneros 39,"Calle del Cardenal Cisneros 39, Madrid España","[40.4341914, -3.7027635]","Calle del Cardenal Cisneros 39, Madrid España","[-3.7027635, 40.4341914]","{'type': 'Point', 'coordinates': [-3.7027635, ...",40.434191,-3.702764
1,61b89d2a5a0da3de9425be09,"La Tagliatella Fuencarral, Madrid","Calle Fuencarral 127, 28010 Madrid España",Italiana,#9.983 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Fuencarral 127,Madrid España,Calle Fuencarral 127,"Calle Fuencarral 127, Madrid España","[40.4333131, -3.7041275]","Calle Fuencarral 127, Madrid España","[-3.7041275, 40.4333131]","{'type': 'Point', 'coordinates': [-3.7041275, ...",40.433313,-3.704127
2,61b89d2a5a0da3de9425be02,Llantén Veggie Bar,"Calle Cardenal Cisneros 40, 28010 Madrid España",Internacional,#1.384 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Cardenal Cisneros 40,Madrid España,Calle Cardenal Cisneros 40,"Calle Cardenal Cisneros 40, Madrid España","[40.4321377, -3.7021397]","Calle Cardenal Cisneros 40, Madrid España","[-3.7021397, 40.4321377]","{'type': 'Point', 'coordinates': [-3.7021397, ...",40.432138,-3.702140
3,61b89d2a5a0da3de9425bdfe,NAP. Neapolitan Authentic Pizza,"Calle Olid 11, 28010 Madrid España",Italiana,#2.349 de 10.409 restaurantes en Madrid,€,Calle Olid 11,Madrid España,Calle Olid 11,"Calle Olid 11, Madrid España","[40.4315985, -3.703331]","Calle Olid 11, Madrid España","[-3.703331, 40.4315985]","{'type': 'Point', 'coordinates': [-3.703331, 4...",40.431598,-3.703331
4,61b89d2a5a0da3de9425bdfd,Kalua Helado Artesanal,"Calle Fuencarral 131 Heladería Kalúa, 28010 Ma...",Confiterías/panaderías,N.º 6 de 477 cafeterías y teterías en Madrid,€,Calle Fuencarral 131 Heladería Kalúa,Madrid España,Calle Fuencarral 131,"Calle Fuencarral 131, Madrid España","[40.4315634, -3.7036872]","Calle Fuencarral 131, Madrid España","[-3.7036872, 40.4315634]","{'type': 'Point', 'coordinates': [-3.7036872, ...",40.431563,-3.703687
5,61b89d2a5a0da3de9425be0c,Bacira,"Calle del Castillo 16, 28010 Madrid España",Japonesa,#301 de 10.409 restaurantes en Madrid,€€ - €€€,Calle del Castillo 16,Madrid España,Calle del Castillo 16,"Calle del Castillo 16, Madrid España","[40.4337811, -3.6989949]","Calle del Castillo 16, Madrid España","[-3.6989949, 40.4337811]","{'type': 'Point', 'coordinates': [-3.6989949, ...",40.433781,-3.698995
6,61b89d2a5a0da3de9425be10,La Espumosa Terraza MADRID,Calle de Santa Engracia 60 Entrada Glorieta de...,Bar,#3.064 de 10.409 restaurantes en Madrid,€€ - €€€,Calle de Santa Engracia 60 Entrada Glorieta de...,Madrid España,Calle de Santa Engracia 60,"Calle de Santa Engracia 60, Madrid España","[40.4353703, -3.6987443]","Calle de Santa Engracia 60, Madrid España","[-3.6987443, 40.4353703]","{'type': 'Point', 'coordinates': [-3.6987443, ...",40.435370,-3.698744
7,61b89d2a5a0da3de9425be0e,Kitchen 154 (Vallehermoso),"Calle de Vallehermoso, 36 Mercado de Valleherm...",India,#391 de 10.409 restaurantes en Madrid,€€ - €€€,Calle de Vallehermoso 36 Mercado de Vallehermo...,Madrid España,Calle de Vallehermoso 36,"Calle de Vallehermoso 36, Madrid España","[40.4340111, -3.7081041]","Calle de Vallehermoso 36, Madrid España","[-3.7081041, 40.4340111]","{'type': 'Point', 'coordinates': [-3.7081041, ...",40.434011,-3.708104
8,61b89d2a5a0da3de9425bdfc,La Tomata,"Calle Alburquerque 13, 28010 Madrid España",Mexicana,#614 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Alburquerque 13,Madrid España,Calle Alburquerque 13,"Calle Alburquerque 13, Madrid España","[40.4311766, -3.7020589]","Calle Alburquerque 13, Madrid España","[-3.7020589, 40.4311766]","{'type': 'Point', 'coordinates': [-3.7020589, ...",40.

In [225]:
map_1 = folium.Map(location=[inicial_lat,inicial_long], zoom_start=15)

dic = {"location": [inicial_lat, inicial_long], "tooltip": "ubicacion"}

icono = Icon(color = "red",
                     prefix="fa",
                     icon="map-marker",
                     icon_color="black")

mark = Marker(**dic, icon=icono)
mark.add_to(map_1)

In [226]:
map_1

In [227]:
df_restaurantes.type.unique()

array(['Mediterránea', 'Italiana', 'Internacional',
       'Confiterías/panaderías', 'Japonesa', 'Bar', 'India', 'Mexicana',
       'Pub con cerveza artesanal'], dtype=object)

In [250]:
for i,row in df_restaurantes.iterrows():

    #dic = {"location": [row["latitud"], row["longitud"]], "tooltip": row["nombre"]}
    dic = {"location": [row["latitud"], row["longitude"]], "tooltip": row["name"]}

    if row["type"] == "Mediterránea":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")

    elif row["type"] == 'Italiana':
        icono = Icon(color = "lightblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")

        
    elif row["type"] == "Internacional":        
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    elif row["type"] == "Confiterías/panaderías":        
        icono = Icon(color = "gray",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    elif row["type"] == "Japonesa":        
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "Bar":        
        icono = Icon(color = "lightgreen",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "India":        
        icono = Icon(color = "lightblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
                 
           
    elif row["type"] == "Mexicana":        
        icono = Icon(color = "pink",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "Pub con cerveza artesanal":        
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    
    mark = Marker(**dic, icon=icono)
    mark.add_to(map_1)   

In [251]:
map_1

1- Input preguntando donde estas
2- Input preguntando que radio quieren
3- Funcion que pasa de direccion a coordenadas (coordenadas al reves)
4- Funcion que me haga una query near a moongo y me devuelva todos los restaurantes en el radio  




streamlit 

1- mapas (aqui va el input del usuario)
para la distancia hago un sidebar. esta en iris_prediction/src/ support.py /
2 - graficos
